In [3]:
#selectie
#ammonium, kwik, vis, cadmium
import numpy as np
import pandas as pd
import geopandas as gpd
from numpy.random import random
from bokeh.io import output_notebook, output_file, save
from bokeh.plotting import figure, show
from bokeh.tile_providers import CARTODBPOSITRON,CARTODBPOSITRON_RETINA
from bokeh.tile_providers import STAMEN_TONER
#from bokeh.models import WMTSTileSource
from bokeh.models import ColumnDataSource, HoverTool, GeoJSONDataSource,  CustomJS, Div
from bokeh.models.widgets import DataTable, TableColumn, HTMLTemplateFormatter, Button, Select, RadioGroup
from bokeh.layouts import column, row, widgetbox

from bokeh.io import export_png

from natsort import natsorted
from shapely.geometry import Point, LineString
from ipywidgets import widgets

debug_view = widgets.Output(layout={'border': '1px solid black'})

oordeel = pd.read_hdf('oordeel2018.h5',key='o2018')

#direct output to notebook by default
output_notebook()
############################# LOAD SHAPEFILES DATA
#setup waterlichamen from centroids in shapefiles
shape_wlpt = gpd.read_file('shapefiles/vlak_centroid.shp', crs={'init': 'epsg:28992'})
shape_wlpt.crs={'init': 'epsg:28992'}
shape_wlpt1 = gpd.read_file('shapefiles/lijn_centroid.shp', crs={'init': 'epsg:28992'})
shape_wlpt1.crs={'init': 'epsg:28992'}
shape_wlpt = pd.concat([shape_wlpt, shape_wlpt1])

# create Amfo X,Y from geometry to allow merging 
shape_wlpt['projectie_x']=shape_wlpt.geometry.x
shape_wlpt['projectie_y']=shape_wlpt.geometry.y

############################# LOAD MONITOR DATA
localdir='datasets/'

datafiles= [#'1.maatregelen_20140507.csv', 
            '4.monitoringprogramma_owl_20181016.csv']

for index in range(len(datafiles)):
    locd=pd.read_csv(localdir+datafiles[index], sep=';', decimal='.', dtype=object, encoding='latin1')
    locd.columns = locd.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('.', '_').str.replace(')', '')

#convert X,Y to floating points (normally obj after loading)
locd.geometriepunt_x_rd=locd.geometriepunt_x_rd.astype(float)
locd.geometriepunt_y_rd=locd.geometriepunt_y_rd.astype(float)
############################# SELECT DATA
#selection

#add all centroid points for projections to database (outer-merge)
merged=pd.merge(locd,shape_wlpt, left_on='representatief_waterlichaam_code', right_on='owmident', how='outer')
#remove any row where X or Y points are NaN
merged = merged[np.isfinite(merged['geometriepunt_x_rd'])]
merged = merged[np.isfinite(merged['geometriepunt_y_rd'])]
merged = merged[np.isfinite(merged['projectie_x'])]
merged = merged[np.isfinite(merged['projectie_y'])]

merged=pd.merge(oordeel,merged,left_on=['waterlichaam_identificatie','typering_omschrijving'],
                right_on=['representatief_waterlichaam_code','parameter_omschrijving'], how='right')

#remap the oordelen to color
omap= { 'oordeel':{ 'slecht':'red','ontoereikend':'orange','matig':'yellow',
                   'goed':'lightgreen','zeer goed':'blue','voldoet niet':'red','voldoet':'blue', np.nan:'white'} }
merged=merged.replace(omap)

#project centroids into webmercator(3857) from amfo (28992)
shape_wlpt.crs={'init': 'epsg:28992'}
shape_wlpt=shape_wlpt.to_crs(epsg=3857)

#collect all parameters
paramselect=merged['typering_omschrijving'].dropna().unique()
params=sorted(paramselect.tolist())
paramselect=merged['waterbeheerder_omschrijving'].dropna().unique()
paramsbeheer=['all']+sorted(paramselect.tolist())
############################# SELECT DATA
def get_data(param, param1, selectie):
    global merged, shape_wlpt, sourcewl, sourcewlM, lijn_source
    #select data
    subset=merged.query('parameter_omschrijving==@param')
    if param1!='all':
        subset=subset.query('waterbeheerder_omschrijving==@param1')
    if selectie==2:
        subset=subset.query('oordeel!="white"')
    ############################# create set of projected circles
    a=subset.copy()
    a=a.query('ligt_in_waterlichaam_code!=representatief_waterlichaam_code')
    testlist=a[['representatief_waterlichaam_code','oordeel', 'waterbeheerder_omschrijving']]
    del a
    shape_wl=gpd.GeoDataFrame(pd.merge(testlist,shape_wlpt,
                                       left_on="representatief_waterlichaam_code",right_on='owmident'))
    sourcewl = ColumnDataSource(data=dict(
                            x=list(shape_wl['geometry'].x), #map geometry
                            y=list(shape_wl['geometry'].y),
                            representatief_waterlichaam_code=list(shape_wl['owmident'] ),
                            representatief_waterlichaam_omschrijving=list(shape_wl['owmnaam']),
                            #colorl=list(shape_wlpt['marker_color']),
                            owmtyped=list(shape_wl['owmtype']),
                            beheerder=list(shape_wl['waterbeheerder_omschrijving']),
                            oordeel=list(shape_wl['oordeel'])
                            ))
    ########################### create set of not projected circles
    a=subset.copy()
    a=a.query('ligt_in_waterlichaam_code==representatief_waterlichaam_code')
    testlist=a[['ligt_in_waterlichaam_code','oordeel','waterbeheerder_omschrijving']]   
    del a
    #testlist=testlist.rename( columns={0: "test"})
    shape_wlM=gpd.GeoDataFrame(pd.merge(testlist,shape_wlpt,
                                        left_on="ligt_in_waterlichaam_code",right_on='owmident'))
    sourcewlM = ColumnDataSource(data=dict(
                            x=list(shape_wlM['geometry'].x), #map geometry
                            y=list(shape_wlM['geometry'].y),
                            representatief_waterlichaam_code=list(shape_wlM['owmident'] ),
                            representatief_waterlichaam_omschrijving=list(shape_wlM['owmnaam']),
                            #colorl=list(shape_wlpt['marker_color']),
                            owmtyped=list(shape_wlM['owmtype']),
                            beheerder=list(shape_wlM['waterbeheerder_omschrijving']),
                            oordeel=list(shape_wlM['oordeel'])
                            ))

    ###########################create a set of lines between meetpunt - projected points and project into webmercator
    if selectie==1:
        subset=gpd.GeoDataFrame(subset.query('ligt_in_waterlichaam_code!=representatief_waterlichaam_code'))
    else:     
        subset=gpd.GeoDataFrame(subset)
        
    subset['geometry'] =  subset.apply(lambda x: LineString([(x['geometriepunt_x_rd'], x['geometriepunt_y_rd'] ) ,
                                                             (x['projectie_x'], x['projectie_y'] )] ),axis=1)
    subset.crs={'init': 'epsg:28992'}
    subset=subset.to_crs(epsg=3857)
    lijn_source = GeoJSONDataSource(geojson=subset.to_json())

bounds=shape_wlpt.total_bounds
get_data('ammonium','all', 0)
########################################### PLOT #####################################################

@debug_view.capture(clear_output=True)
def modify_doc(doc):
    #initial dataset
    print('doc')
    hover = HoverTool(tooltips=[
    ("waterlichaam", "@representatief_waterlichaam_code"),
    ("info", "@representatief_waterlichaam_omschrijving"),
    ("typering","@owmtyped"),
    ("meetpunt","@ligt_in_waterlichaam_code"),
    ("info","@meetpunt_omschrijving"),
     ("parameter","@parameter_omschrijving"),
        ("beheerder","@beheerder")
     ])
    
    #sourcept = ColumnDataSource(data=get_data('ammonium'))
    sourcept=sourcewl
    sourceptM=sourcewlM
    sourcelijn=lijn_source
    #sourcebestrijding=sourcebes
    
    p = figure(match_aspect=True, aspect_scale=1, title='Waterlichaam', tools=[hover, "box_zoom, reset, pan,lasso_select,box_select, wheel_zoom,tap"],
                x_range=(bounds[0], bounds[2]), y_range=(bounds[1], bounds[3]),
                 x_axis_type="mercator", y_axis_type="mercator", width=900)

    p.xaxis.visible = False
    p.yaxis.visible = False
    p.add_tile(CARTODBPOSITRON, alpha=0.6)
      
    p.circle(x='x', y='y', size=6, source=sourcept, legend='projectie', fill_color='white',
                 color='oordeel', line_width=2,alpha=0.8)

    p.circle(x='x', y='y', source=sourceptM, size=7, color='lightgray', fill_color='oordeel',
          #line_color='colorl', fill_color='colorl',
                        line_width=1, fill_alpha=0.9, legend='waterlichaam')
    
    project=p.multi_line('xs', 'ys', source=sourcelijn,  line_width=1, legend= "projectie-lijn")
    project.visible=False
    
    p.legend.location = "top_left"
    p.legend.click_policy="hide"

    select = Select(title="Parameter", value="ammonium", options=params, width=100, height=40)
    selectbeheer = Select(title="Beheerder", value="all", options=paramsbeheer, width=400, height=40)
    radio_group = RadioGroup(height=60,
        labels=["alles", "projecties","not empty"], active=0)
    
    button = Button(label="Save", width=50, height=30)
    def save_data():
        filen="{0}.png".format(select.value)
        export_png(p,filename=filen)
    button.on_click(save_data)
    
    def update_selectie(attrname, old, new):
        get_data(select.value, selectbeheer.value, radio_group.active)
        #metingen
        sourcept.data=sourcewl.data
        sourceptM.data=sourcewlM.data
        #sourcebestrijding.data=sourcebes.data
        sourcelijn.geojson=lijn_source.geojson
        p.title.text ="stof: {0} beheerders: {1}".format(select.value, selectbeheer.value)       
        #select.title = radio_group.active
        export_png(p,'lastplot.png')
    
    select.on_change('value', update_selectie)
    selectbeheer.on_change('value', update_selectie)
    radio_group.on_change('active', update_selectie)
    layout = column(row(radio_group), button, row(selectbeheer,select, width=400), p,  width=800)
        
    doc.add_root(layout)

display(debug_view)    
show(modify_doc)

Loading BokehJS ...

Output(layout=Layout(border='1px solid black'))

In [62]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101213 entries, 0 to 101212
Data columns (total 45 columns):
typering_omschrijving                            62012 non-null object
oordeelsoort_code                                62012 non-null category
oordeel                                          101213 non-null object
waardebepalingsmethode_code                      62012 non-null object
waterlichaam_identificatie                       62012 non-null object
meetpunt_identificatie                           101213 non-null object
meetpunt_omschrijving                            101213 non-null object
geometriepunt_x_rd                               101213 non-null float64
geometriepunt_y_rd                               101213 non-null float64
parameter_code                                   101213 non-null object
parameter_omschrijving                           100468 non-null object
cas_nr                                           100468 non-null object
monitoring_soort         

In [40]:
#geprojecteerd
a=pd.DataFrame(sourcewl.data)
a=a[['oordeel','representatief_waterlichaam_code']].drop_duplicates()
a1=a.groupby('oordeel')['representatief_waterlichaam_code'].count()
#niet geprojecteerd
b=pd.DataFrame(sourcewlM.data)
b1=b[['oordeel','representatief_waterlichaam_code']].drop_duplicates()

b1=b1.groupby('oordeel')['representatief_waterlichaam_code'].count()
print(b1) #number of waterbodies with measurement inside
c=a.loc[~a.index.isin(a.merge(b.assign(a='key'),how='left').dropna().index)]
c1=c.groupby('oordeel')['representatief_waterlichaam_code'].count()
print(c1) #number of waterbodies with measurement ONLY out
d=pd.concat([b,c], sort=False).drop_duplicates()
d1=d.groupby('oordeel')['representatief_waterlichaam_code'].count()
d1
p1=figure(width=300)
d1
a=pd.DataFrame(sourcewlM.data).drop_duplicates()
a.query('oordeel=="red"')

oordeel
blue     17
red      19
white    34
Name: representatief_waterlichaam_code, dtype: int64
oordeel
blue     105
red      127
white     13
Name: representatief_waterlichaam_code, dtype: int64


,x,y,representatief_waterlichaam_code,representatief_waterlichaam_omschrijving,owmtyped,oordeel
0,493683.506130,6.747899e+06,NL94_1,Haringvliet-oost,R8,red
1,520108.564677,6.780240e+06,NL94_4,Oude Maas,R8,red
3,516780.886900,6.792815e+06,NL94_7,Hollandsche IJssel,R8,red
5,680366.622910,6.850747e+06,NL93_IJSSEL,IJssel,R7,red
6,623151.498943,6.781924e+06,NL93_8,"Bovenrijn, Waal",R7,red
7,598344.209319,6.947892e+06,NL92_IJSSELMEER,IJsselmeer,M21,red
8,644887.954497,6.905766e+06,NL92_KETELMEER_VOSSEMEER,"Ketelmeer, Vossemeer",M14,red
9,585040.570136,6.892609e+06,NL92_MARKERMEER,Markermeer,M21,red
17,469676.041488,6.738903e+06,NL89_volkerak,Volkerak,M20,red
18,556844.624557,6.828146e+06,NL86_6,Amsterdam-Rijnkanaal Noordpand,M7b,red


In [63]:
oordeel.head()

,typering_omschrijving,oordeelsoort_code,oordeel,waardebepalingsmethode_code,waterlichaam_identificatie
440,"1,2-dichloorethaan",TT_OM,voldoet,AQUOKIT,NL99_BRA_02_3E
442,"4,4'-dichloordifenyltrichloorethaan",TT_OM,voldoet,AQUOKIT,NL99_BRA_02_3E
444,antraceen,TT_OM,voldoet,AQUOKIT,NL99_BRA_02_3E
446,benzo(b)fluorantheen,TT_OM,voldoet,AQUOKIT,NL99_BRA_02_3E
448,benzeen,TT_OM,voldoet,AQUOKIT,NL99_BRA_02_3E
